# APPLIED DATA SCIENCE CAPSTONE

In [157]:
import pandas as pd
import numpy as np
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### First we need to import the data obtained from https://www.kaggle.com/nikitagrec/world-capitals-gps

In [187]:
Data=pd.read_csv('concap.csv')
Data.head()

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,ContinentName
0,Somaliland,Hargeisa,9.550000,44.050000,NaN,Africa
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS,Antarctica
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,TF,Antarctica
3,Palestine,Jerusalem,31.766667,35.233333,PS,Asia
4,Aland Islands,Mariehamn,60.116667,19.900000,AX,Europe


### Now let's filtrate the data because we only need European capital cities, and also drop the columns that we don't need

In [189]:
Data=Data[Data["ContinentName"]=="Europe"]
Data.drop(columns=["CountryName", "CountryCode","ContinentName"], inplace=True)
print("There are", Data.shape[0], "Cities in the data")
Data.head()

There are 57 Cities in the data


,CapitalName,CapitalLatitude,CapitalLongitude
4,Mariehamn,60.116667,19.900000
10,Tirana,41.316667,19.816667
13,Andorra la Vella,42.500000,1.516667
18,Yerevan,40.166667,44.500000
21,Vienna,48.200000,16.366667


### Let's plot a map with the location of the cities to have an idea of the geographical distribution

In [212]:

map_world = folium.Map(location=[50.91,23.52],zoom_start=4)

# add markers to map
for lat, lng, capital in zip(Data['CapitalLatitude'], Data['CapitalLongitude'], Data['CapitalName']):
    label = '{}, {}'.format(capital, country)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_world)  
    
map_world

### Foursquare credentials

In [222]:
CLIENT_ID = 'HFSRRHRGQKWKE45BO1AD0HNV3V3H0BG4FHRYERH5HDY5LVO4' 
CLIENT_SECRET = 'IMIGNU3BPFMXYQEKX2LTIRO3CSA43L5ZVDJJHLJE5DQ21R5P' 
VERSION = '20180605' 
LIMIT=150

#### This function will help us to get the venues of every city in a radius of 10km

In [223]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City lLatitude', 
                  'CityLongitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [224]:
Capital_Venues = getNearbyVenues(names=Data['CapitalName'],
                                   latitudes=Data['CapitalLatitude'],
                                   longitudes=Data['CapitalLongitude']
                                  )

Mariehamn
Tirana
Andorra la Vella
Yerevan
Vienna
Baku
Minsk
Brussels
Sarajevo
Sofia
Zagreb
Nicosia
Prague
Copenhagen
Tallinn
Torshavn
Helsinki
Paris
Tbilisi
Berlin
Gibraltar
Athens
Saint Peter Port
Vatican City
Budapest
Reykjavik
Dublin
Douglas
Rome
Saint Helier
Pristina
Riga
Vaduz
Vilnius
Luxembourg
Skopje
Valletta
Chisinau
Monaco
Podgorica
Amsterdam
Oslo
Warsaw
Lisbon
Bucharest
Moscow
Belgrade
Bratislava
Ljubljana
Madrid
Longyearbyen
Stockholm
Bern
Ankara
Kyiv
London
North Nicosia


In [225]:
Capital_Venues.head()

,City,City lLatitude,CityLongitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mariehamn,60.116667,19.9,Ålands Sjöfartsmuseum,60.097170,19.926457,Museum
1,Mariehamn,60.116667,19.9,ÅSS Marina,60.098890,19.924264,Harbor / Marina
2,Mariehamn,60.116667,19.9,Bagarstugan Cafe & Vin,60.099727,19.943189,Café
3,Mariehamn,60.116667,19.9,Indigo,60.100092,19.942963,Lounge
4,Mariehamn,60.116667,19.9,Sjökvarteret,60.104880,19.945174,History Museum


#### Now let's see which are the more commons Venues and how many cities have one of them

In [226]:
disc=[]
for venue in Capital_Venues.groupby("Venue Category").count().index:
    if Capital_Venues[Capital_Venues["Venue Category"]==venue].iloc[:,0].value_counts().shape[0] >= 43:
        disc.append([venue,Capital_Venues[Capital_Venues["Venue Category"]==venue].iloc[:,0].value_counts().shape[0]])

Discard=pd.DataFrame(disc, columns=["Venue Category","City Count"])
Discard.sort_values(by=["City Count"],ascending=False)

,Venue Category,City Count
1,Café,54
3,Hotel,52
4,Park,51
2,Coffee Shop,48
0,Bakery,45
5,Restaurant,45


#### now let's delete this venue categories. (The explanation of this is in the report)

In [227]:
for venue in list(Discard["Venue Category"]):
    Capital_Venues.drop(Capital_Venues[Capital_Venues["Venue Category"]==venue].index,axis=0,inplace=True)

Now let's see which are the most common venues for each neighbourhood

In [228]:
# one hot encoding
Capitals_onehot = pd.get_dummies(Capital_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Capitals_onehot['City'] = Capital_Venues['City'] 

# move neighborhood column to the first column
fixed_columns = [Capitals_onehot.columns[-1]] + list(Capitals_onehot.columns[:-1])
Capitals_onehot = Capitals_onehot[fixed_columns]

Capitals_onehot.head()

,Zoo,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Antique Shop,...,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [229]:
Capital_grouped = Capitals_onehot.groupby('City').mean().reset_index()
Capital_grouped.head()

,City,Zoo,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,...,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Amsterdam,0.013889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.027778,0.013889,0.0,0.0,0.000000,0.027778
1,Andorra la Vella,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.016393,0.000000,0.0,0.0,0.000000,0.000000
2,Ankara,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.011905,0.011905
3,Athens,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.039474,0.000000,0.0,0.0,0.000000,0.000000
4,Baku,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000


This Function will help us to get the most common venues of each Neighbourood

In [230]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [231]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

    
# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = Capital_grouped['City']

for ind in np.arange(Capital_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Capital_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amsterdam,Ice Cream Shop,French Restaurant,Pizza Place,Plaza,Bar,Breakfast Spot,Cocktail Bar,Cheese Shop,Music Venue,Japanese Restaurant
1,Andorra la Vella,Spanish Restaurant,BBQ Joint,Resort,Mediterranean Restaurant,Bar,Ski Area,Pizza Place,Sporting Goods Shop,Gym / Fitness Center,Falafel Restaurant
2,Ankara,History Museum,Theater,Turkish Restaurant,Seafood Restaurant,Dance Studio,Comfort Food Restaurant,Art Gallery,Camera Store,Sandwich Place,Dessert Shop
3,Athens,Historic Site,Dessert Shop,Souvlaki Shop,Bar,History Museum,Gourmet Shop,Wine Bar,Boutique,Taverna,Italian Restaurant
4,Baku,Plaza,Spa,Lounge,Supermarket,Big Box Store,Italian Restaurant,Turkish Restaurant,Shopping Mall,Historic Site,Cupcake Shop


Now it's time to run the k-means classification model

In [207]:
# set number of clusters
kclusters =6

Capital_grouped_clustering = Capital_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Capital_grouped_clustering)

kmeans.labels_

array([2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 4, 4, 4, 2, 2, 5, 2, 2,
       0, 1, 5, 1, 2, 2, 2, 0, 0, 0, 5, 1, 2, 0, 0, 2, 3, 4, 4, 2, 1, 2,
       2, 2, 2, 2, 0, 1, 1, 3, 5, 2, 2, 1, 2])

Whit this code we merge the cluster array with the main dataframe

In [232]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Capitals_merged = Data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Capitals_merged = Capitals_merged.join(city_venues_sorted.set_index('City'), on='CapitalName')

Capitals_merged.head(6)# check the last columns!


,CapitalName,CapitalLatitude,CapitalLongitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Mariehamn,60.116667,19.900000,1,Bar,Supermarket,Lounge,Pizza Place,History Museum,Museum,Fast Food Restaurant,Campground,Canal,Gym / Fitness Center
10,Tirana,41.316667,19.816667,2,Eastern European Restaurant,Italian Restaurant,Cocktail Bar,Fast Food Restaurant,Lounge,Ice Cream Shop,Pizza Place,Mediterranean Restaurant,Clothing Store,Seafood Restaurant
13,Andorra la Vella,42.500000,1.516667,1,Spanish Restaurant,BBQ Joint,Resort,Mediterranean Restaurant,Bar,Ski Area,Pizza Place,Sporting Goods Shop,Gym / Fitness Center,Falafel Restaurant
18,Yerevan,40.166667,44.500000,1,Supermarket,Pool,Comfort Food Restaurant,Plaza,Caucasian Restaurant,Gym,Gym Pool,Italian Restaurant,Eastern European Restaurant,Monument / Landmark
21,Vienna,48.200000,16.366667,5,Plaza,Austrian Restaurant,Ice Cream Shop,Palace,Pizza Place,Church,Concert Hall,Vietnamese Restaurant,Fountain,Monument / Landmark
22,Baku,40.383333,49.866667,1,Plaza,Spa,Lounge,Supermarket,Big Box Store,Italian Restaurant,Turkish Restaurant,Shopping Mall,Historic Site,Cupcake Shop


Now we drop any null value

In [233]:
Capitals_merged.dropna(subset=["Cluster Labels"],axis=0,inplace=True)
Capitals_merged["Cluster Labels"]=Capitals_merged["Cluster Labels"].astype(int)
Capitals_merged.head(6)

,CapitalName,CapitalLatitude,CapitalLongitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Mariehamn,60.116667,19.900000,1,Bar,Supermarket,Lounge,Pizza Place,History Museum,Museum,Fast Food Restaurant,Campground,Canal,Gym / Fitness Center
10,Tirana,41.316667,19.816667,2,Eastern European Restaurant,Italian Restaurant,Cocktail Bar,Fast Food Restaurant,Lounge,Ice Cream Shop,Pizza Place,Mediterranean Restaurant,Clothing Store,Seafood Restaurant
13,Andorra la Vella,42.500000,1.516667,1,Spanish Restaurant,BBQ Joint,Resort,Mediterranean Restaurant,Bar,Ski Area,Pizza Place,Sporting Goods Shop,Gym / Fitness Center,Falafel Restaurant
18,Yerevan,40.166667,44.500000,1,Supermarket,Pool,Comfort Food Restaurant,Plaza,Caucasian Restaurant,Gym,Gym Pool,Italian Restaurant,Eastern European Restaurant,Monument / Landmark
21,Vienna,48.200000,16.366667,5,Plaza,Austrian Restaurant,Ice Cream Shop,Palace,Pizza Place,Church,Concert Hall,Vietnamese Restaurant,Fountain,Monument / Landmark
22,Baku,40.383333,49.866667,1,Plaza,Spa,Lounge,Supermarket,Big Box Store,Italian Restaurant,Turkish Restaurant,Shopping Mall,Historic Site,Cupcake Shop


Now we can plot a map with the cities and clusters

In [234]:
# create map
map_clusters =folium.Map(location=[50.91,23.52],zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Capitals_merged['CapitalLatitude'], Capitals_merged['CapitalLongitude'], Capitals_merged['CapitalName'], Capitals_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Finally, let's see the most common venues in each cluster

In [235]:
Capitals_merged[Capitals_merged["Cluster Labels"]==0].head()

,CapitalName,CapitalLatitude,CapitalLongitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Nicosia,35.166667,33.366667,0,Greek Restaurant,Wine Bar,Bar,Sandwich Place,Gym,Turkish Restaurant,Beer Garden,Seafood Restaurant,Gym / Fitness Center,Grocery Store
78,Torshavn,62.000000,-6.766667,0,Scandinavian Restaurant,Steakhouse,Pier,Sushi Restaurant,Fish & Chips Shop,Tapas Restaurant,Beer Bar,Bar,Museum,Performing Arts Venue
102,Reykjavik,64.150000,-21.950000,0,Bar,Seafood Restaurant,Burger Joint,Scandinavian Restaurant,Record Shop,Pool,Beer Bar,Men's Store,Concert Hall,Bookstore
120,Pristina,42.666667,21.166667,0,Bar,Mediterranean Restaurant,Eastern European Restaurant,Italian Restaurant,Diner,Nightclub,Shopping Mall,Bistro,Steakhouse,Comfort Food Restaurant
162,Oslo,59.916667,10.750000,0,Bar,Burger Joint,Movie Theater,Scandinavian Restaurant,Vegetarian / Vegan Restaurant,Brewery,French Restaurant,Bookstore,Scenic Lookout,Gourmet Shop


In [236]:
Capitals_merged[Capitals_merged["Cluster Labels"]==1].head()

,CapitalName,CapitalLatitude,CapitalLongitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Mariehamn,60.116667,19.900000,1,Bar,Supermarket,Lounge,Pizza Place,History Museum,Museum,Fast Food Restaurant,Campground,Canal,Gym / Fitness Center
13,Andorra la Vella,42.500000,1.516667,1,Spanish Restaurant,BBQ Joint,Resort,Mediterranean Restaurant,Bar,Ski Area,Pizza Place,Sporting Goods Shop,Gym / Fitness Center,Falafel Restaurant
18,Yerevan,40.166667,44.500000,1,Supermarket,Pool,Comfort Food Restaurant,Plaza,Caucasian Restaurant,Gym,Gym Pool,Italian Restaurant,Eastern European Restaurant,Monument / Landmark
22,Baku,40.383333,49.866667,1,Plaza,Spa,Lounge,Supermarket,Big Box Store,Italian Restaurant,Turkish Restaurant,Shopping Mall,Historic Site,Cupcake Shop
89,Athens,37.983333,23.733333,1,Historic Site,Dessert Shop,Souvlaki Shop,Bar,History Museum,Gourmet Shop,Wine Bar,Boutique,Taverna,Italian Restaurant


In [237]:
Capitals_merged[Capitals_merged["Cluster Labels"]==2].head()

,CapitalName,CapitalLatitude,CapitalLongitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Tirana,41.316667,19.816667,2,Eastern European Restaurant,Italian Restaurant,Cocktail Bar,Fast Food Restaurant,Lounge,Ice Cream Shop,Pizza Place,Mediterranean Restaurant,Clothing Store,Seafood Restaurant
27,Minsk,53.900000,27.566667,2,Bar,Spa,Plaza,Wine Bar,Waterfront,Italian Restaurant,Gym / Fitness Center,Arcade,French Restaurant,Salon / Barbershop
28,Brussels,50.833333,4.333333,2,Plaza,Bar,Concert Hall,French Restaurant,Pizza Place,Supermarket,Beer Store,Organic Grocery,Sandwich Place,Flower Shop
34,Sarajevo,43.866667,18.416667,2,Eastern European Restaurant,Italian Restaurant,Dessert Shop,Tea Room,Fast Food Restaurant,Mountain,Mediterranean Restaurant,Theme Park,Castle,Multiplex
39,Sofia,42.683333,23.316667,2,Vegetarian / Vegan Restaurant,Plaza,Yoga Studio,Dessert Shop,Cocktail Bar,Clothing Store,Cheese Shop,Bookstore,Bistro,Beer Store


In [238]:
Capitals_merged[Capitals_merged["Cluster Labels"]==3].head()

,CapitalName,CapitalLatitude,CapitalLongitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
99,Vatican City,41.9,12.450000,3,Ice Cream Shop,Pizza Place,Italian Restaurant,Plaza,Sandwich Place,Historic Site,Diner,Scenic Lookout,Monument / Landmark,Garden
110,Rome,41.9,12.483333,3,Ice Cream Shop,Pizza Place,Historic Site,Italian Restaurant,Sandwich Place,Plaza,Diner,Scenic Lookout,Monument / Landmark,Art Museum


In [239]:
Capitals_merged[Capitals_merged["Cluster Labels"]==4].head()

,CapitalName,CapitalLatitude,CapitalLongitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
88,Gibraltar,36.133333,-5.350000,4,Tapas Restaurant,Pub,Spanish Restaurant,Beach,Mediterranean Restaurant,Indian Restaurant,Steakhouse,Italian Restaurant,Plaza,Bar
94,Saint Peter Port,49.450000,-2.533333,4,Pub,Bar,Department Store,Supermarket,Beach,Seafood Restaurant,Steakhouse,Nightclub,Thai Restaurant,Castle
107,Dublin,53.316667,-6.233333,4,Pub,Plaza,Beach,Stadium,Pizza Place,Italian Restaurant,Tea Room,Lighthouse,Gastropub,Zoo
108,Douglas,54.150000,-4.483333,4,Pub,Gastropub,Racetrack,Bar,Italian Restaurant,Gym / Fitness Center,Seafood Restaurant,Department Store,Movie Theater,Bookstore
113,Saint Helier,49.183333,-2.100000,4,Pub,Department Store,Bar,Gastropub,Supermarket,Breakfast Spot,Seafood Restaurant,Pizza Place,Fish & Chips Shop,Garden Center


In [240]:
Capitals_merged[Capitals_merged["Cluster Labels"]==5].head()

,CapitalName,CapitalLatitude,CapitalLongitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Vienna,48.200000,16.366667,5,Plaza,Austrian Restaurant,Ice Cream Shop,Palace,Pizza Place,Church,Concert Hall,Vietnamese Restaurant,Fountain,Monument / Landmark
81,Paris,48.866667,2.333333,5,Plaza,Garden,Art Museum,French Restaurant,Wine Bar,Bookstore,Historic Site,Cheese Shop,Indie Movie Theater,Pedestrian Plaza
173,Lisbon,38.716667,-9.133333,5,Plaza,Scenic Lookout,Garden,Portuguese Restaurant,Ice Cream Shop,Museum,Breakfast Spot,Gym,Burger Joint,Vegetarian / Vegan Restaurant
200,Madrid,40.400000,-3.683333,5,Plaza,Art Museum,Art Gallery,Spanish Restaurant,Tapas Restaurant,Monument / Landmark,Theater,Hostel,Garden,Japanese Restaurant
